Working with videos under fast.ai and pytorch
tl;dr: When working with visual data (images, video) a key element is data augmentation. Afaik right now, fastai provides superior support than pytorch and even keras/tf. I want to apply these augmentations to videos too. Also, this uses fastai 1.0.19. Things move fast so there might be easyer ways to do it.

Generics
First constraint is that 3D convolutions require data in  (𝑁,𝐶,𝐷,𝐻,𝑊)  format where N is the batch size (ignoring it for now) C is channel color, D is the time dimension (number of frames) and H,W are spatial dimensions.

2nd constraint is that all the image transformations routines (vision.transform) accept  (𝐶,𝐻,𝑊)  images, without a boundary on how large C is. So convert your video into  (𝐶,𝐻,𝑊)  where  𝐶=3×𝐷 .

One approach is to write your own derived vision.ImageDataset class that takes care of almost everything. See Video classification forum post. But in 1.0.19 one can "plug in" his/her own image_opener inside the standard image dataset.

Here is another approach:
Use Opencv + ImageClassificationBase.image_opener to load a video in vision.Image() as a 3D tensor  (𝐶,𝑊,𝐻) 
Generate regular data augmentation operations
Construct a DatasetTfm and use it in your models
Convert from 3D to 4D tensor, just before entering the model
Notes
IMPORTANT NOTE. I struggled for several hours to convert step 3 into a proprer Transform() operation, that can be added to the tfms list. Maybe smb can do it.

In general, Transform operations could handle arbitrary size tensors, where the only assumption is that the last 2 plaes are (W, H). I did not dig deeper now, but in an earlier version there was a line where the tensor shape was unpacked to three values.

Videos of arbitrary size will have different  𝐷  dimensions and they will not be collated into a batch. Here, I just read first 32 frames of each video. One can either code a time augmentation (eg random crop of fixed size) when the video is opened or, better, create a Transform that will do the time-aware cropping.

Using opencv removes the need of offline splitting the videos in frames. (eg using ffmpeg)

Hope it helps!